<table>
    <tr>
        <td><img src='../SystemLink_icon.png' /></td>
        <td ><h1><strong>NI SystemLink Analysis Automation Example - Placeholder</strong></h1></td>
    </tr>
</table>

## Example placeholder
***
Final version will be added soon.
***